<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Cadenas%20b%C3%A1sicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Las cadenas en LangChain representan una secuencia de operaciones que procesan datos de entrada a través de uno o más modelos de lenguaje. Cada operación de una cadena puede considerarse como un paso de un flujo de trabajo, donde el resultado de un paso sirve como entrada para el siguiente. Estas cadenas pueden ser simples o complejas, según los requisitos del proyecto.

Hay varios tipos de cadenas en LangChain:

- Cadenas secuenciales : una disposición lineal de tareas donde el resultado de una tarea alimenta directamente a la siguiente.
- Cadenas paralelas : tareas que pueden realizarse simultáneamente, lo que permite un procesamiento más rápido.
- Cadenas condicionales : cadenas que ejecutan diferentes rutas en función de la entrada proporcionada.

## Nuevas cadenas con LangChain Expression Language (LCEL)
---

En octubre de 2023, LangChain introdujo una nueva forma de trabajar con cadenas, más eficiente y simple, utilizando el operador |.  

Esto permite encadenar elementos de manera intuitiva, con soporte para stream, batch y operaciones asíncronas.  

Además, con LCEL (LangChain Expression Language), las cadenas se pueden definir de forma declarativa y funcional, eliminando la necesidad de clases complejas.   

Aunque las cadenas "antiguas" siguen funcionando, es recomendable adoptar esta nueva sintaxis.

### Ventajas del nuevo sistema:

- **Alineación con el framework:** La nueva sintaxis es el estándar recomendado para futuros proyectos.
- **Interface unificada:** Simplifica la integración de múltiples métodos.
- **Facilidad de composición:** Permite estructurar cadenas de forma secuencial, paralela o con fallbacks.
- **Código más limpio:** Menos líneas para lograr los mismos resultados.

## Configuración del entorno del cuaderno
---

Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).  
1. Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google y Hugging Face.

2. Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

3. Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.

In [ ]:
# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
!pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
!pip install langchain-openai -qU
!pip install langchain-groq -qU
!pip install langchain-google-genai -qU
!pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate


# Importamos las clases necesarias para trabajar con cadenas
from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

El modelo mas básico de cadena seria  PROMPT | LLM

En este la cadena conecta un prompt con un modelo.

Se usa | para conectar el prompt con el modelo. Esto forma una cadena (LLMChain) que toma un texto, lo pasa al prompt y luego al modelo.

Luego la cadena se invoca pasandole el PrompTemplate.

Esto es basicamente lo que hemos venido haciendo hasta ahora sin cadenas.

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Definimos los componentes
prompt = PromptTemplate(template="Traduce esto al {idioma}: {texto}", input_variables=["idioma","texto"])

# Usamos LCEL para crear la cadena
chain = prompt | llm

# Ejecutamos la cadena
# El input del metodo .invoke de una cadena debe ser un unico
# asi que puede hacerse asi,
#     respuesta = chain.invoke("Hola, ¿cómo estás?")
# pero si hay mas de un parametro se espera un diccionario

respuesta = chain.invoke({"idioma":"Francés", "texto":"Hola, ¿cómo estás?"})
display(Markdown(respuesta.content))

Observese que esto es lo que hemos hecho hasta ahora sin cadenas... invocar un LLM pasandole un PromptTemplate formateado...

Aunque no hay gran diferencia en un ejemplo tan sencillo...

La diferencia principal es que:

1. No hay abstracción del prompt como objeto
2. Tenemos que formatear el texto del prompt manualmente
3. No hay una estructura que maneje los inputs de forma estandarizada

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Creamos el prompt y lo formateamos
prompt_template = PromptTemplate(template="Traduce esto al {idioma}: {texto}", input_variables=["idioma","texto"])
prompt_formateado = prompt_template.format(idioma="Francés", texto="Hola, ¿cómo estás?")

# Invocamos el LLM directamente
respuesta = llm.invoke(prompt_formateado)

display(Markdown(respuesta.content))

Veamos unos ejemplos sencillos de tres elementos del tipo PROMPT | LLM | TRANSFORMACION.

Observa que la funcion de transformacion recibe la salida del LLM que es un bjeto AIMessage. Asi que debe acceder al .content de este (que es realmente el string con la respuesta) para poder operar

In [ ]:
# Definimos el PromptTemplate
prompt_template = PromptTemplate.from_template("Responde como si fueras un experto en {tema}: {pregunta}")

# Recuerda que esto es quivalente, con el constructor de clase
# prompt_template = PromptTemplate(
#     template="Responde como si fueras un experto en {tema}: {pregunta}",
#     input_variables=["tema", "pregunta"]
# )


# Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Definimos una función de transformación
def transformar_texto(output):
    return output.content.upper()

# Creamos la cadena con LCEL
chain = prompt_template | llm | transformar_texto

# Ejecutamos la cadena
respuesta = chain.invoke({"tema": "Machine Learning", "pregunta": "¿Qué es el overfitting?"})
display(Markdown(respuesta))

Mejoremos un poco el codigo y observemos la modularidad de la cadenas, reutilizando facilmnente la misma cadena en un bucle de preguntas.

Vamos a hacer uso de un OutputParser. Mas adelante los explicaremos en detalle.

En LangChain, los **parsers de salida** son componentes que se encargan de **transformar y estructurar la salida** generada por un modelo de lenguaje (LLM) o una cadena (chain). Su función principal es asegurarse que la salida esté en un formato específico y utilizable, como un string, un objeto JSON, una lista, etc.  

#### **Tipos comunes de parsers de salida**

- **`StrOutputParser`**: Convierte la salida en un string. (Usaremos este)
    
- **`JsonOutputParser`**: Convierte la salida en un objeto JSON.
    
- **`CommaSeparatedListOutputParser`**: Convierte la salida en una lista separada por comas.
    
- **`StructuredOutputParser`**: Permite definir un esquema personalizado para la salida.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 1. Definimos el PromptTemplate
prompt_template = PromptTemplate.from_template("Responde como si fueras un experto en {tema}: {pregunta}")

# 2. Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# 3. Definimos un OutputParser
# StrOutputParser convierte la salida del LLM en un string.
output_parser = StrOutputParser()

# 4. Definimos una función de transformación
# Explicación: Esta función toma la salida del OutPutParse y la convierte a mayúsculas.
def transformar_texto(output):
    return output.upper()

# 5. Creamos la cadena con LCEL
chain = prompt_template | llm | output_parser | transformar_texto

# 6. Ejecutamos la cadena con diferentes entradas
# Observa cómo la cadena puede reutilizarse para diferentes preguntas.
preguntas = [
    {"tema": "Machine Learning", "pregunta": "¿Qué es el overfitting?"},
    {"tema": "Historia", "pregunta": "¿Quién fue Napoleón Bonaparte?"},
    {"tema": "Programación", "pregunta": "¿Qué es Python?"}
]

for pregunta in preguntas:
    respuesta = chain.invoke(pregunta)
    display(Markdown(f"**Pregunta:** {pregunta['pregunta']}"))
    display(Markdown(f"**Respuesta:** {respuesta}"))
    print("---")  # Separador entre preguntas

Veamos otro ejemplo de cadena: PROMPT | LLM | OutputParser | Guardar en archivo
Esta vez, para una mayor correcion, aunque no es necesario, vamos a usar ChatPromptTemplate

In [ ]:
# Definimos el PromptTemplate
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente conciso y directo. Proporcionas las listas que se te piden y solo la lista sin numerar y en una columna. No añadas ningun tipo de comentario"),
    ("human", "Crea una lista de {numero_elementos} {tipo_elementos}"),
])

# Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# Definimos un OutputParser
output_parser = StrOutputParser()

# Definimos una función para guardar en archivo
def guardar_en_archivo(output):
    with open("respuesta.txt", "w") as f:
        f.write(output)
        print(output)
    return "Respuesta guardada en 'respuesta.txt'"

# Creamos la cadena con LCEL
chain = prompt_template | llm | output_parser | guardar_en_archivo

# Ejecutamos la cadena
respuesta = chain.invoke({"numero_elementos": 10, "tipo_elementos": "Adbervios de modo que no acaben en -mente"})
display(Markdown(respuesta))

Hagamos ahora una Cadena de 3 elementos (Prompt + LLM + OutputParser)
En este caso, usamos un OutputParser que crearemos nosotros mediante una funcion personalizada para estructurar la salida del LLM en un formato específico, como un diccionario.



In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente conciso y directo experto en {tema}. Devuelve el resultado en formato JSON con las claves 'respuesta' y 'explicacion'. No añadas ningun tipo de comentario"),
    ("human", "{pregunta}"),
])

# Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)


json_parser = SimpleJsonOutputParser()


# Creamos la cadena con LCEL
chain = prompt_template | llm | json_parser

# Ejecutamos la cadena
result = chain.invoke({"tema": "Python", "pregunta": "¿Qué es el Streamlit?"})
print(result)

si el modelo ya da la respuesta en json (se le ha indicado en el prompt) porque se se utiliza json_parser = SimpleJsonOutputParser() ????


La respuesta tiene que ver con cómo funcionan los modelos de lenguaje y cómo se manejan sus salidas en LangChain. Vamos a desglosarlo:

---

### 1\. **El modelo devuelve una cadena de texto**

Aunque le indiques al modelo que genere una respuesta en formato JSON, la salida del modelo sigue siendo una **cadena de texto** (un `string`). Por ejemplo, el modelo podría devolver algo como esto:

python

Copy

'{"respuesta": "Streamlit es una biblioteca de Python para crear aplicaciones web interactivas.", "explicacion": "Permite a los desarrolladores crear interfaces de usuario rápidamente utilizando scripts de Python."}'

Esta cadena de texto **no es un objeto JSON** en sí misma, sino una representación textual de un JSON. Para poder trabajar con ella en Python (por ejemplo, acceder a las claves `respuesta` y `explicacion`), necesitas convertirla en un diccionario de Python.

---

### 2\. **¿Qué hace `SimpleJsonOutputParser`?**

`SimpleJsonOutputParser` se encarga de:

1. **Tomar la cadena de texto** que el modelo ha generado.
    
2. **Convertirla en un objeto JSON** (por ejemplo, un diccionario o una lista) utilizando `json.loads()` internamente.
    
3. **Devolver el objeto JSON** para que puedas manipularlo fácilmente en Python.
    

En otras palabras, **`SimpleJsonOutputParser` convierte la cadena JSON en un diccionario de Python**.

---

### 3\. **¿Por qué no se puede usar directamente la salida del modelo?**

Si no usas `SimpleJsonOutputParser` (o algo similar), la salida del modelo sería una cadena de texto, no un diccionario. Por ejemplo:

```
output \= '{"respuesta": "Streamlit es una biblioteca de Python para crear aplicaciones web interactivas.", "explicacion": "Permite a los desarrolladores crear interfaces de usuario rápidamente utilizando scripts de Python."}'
```

Si intentas acceder a `output["respuesta"]`, obtendrás un error, porque `output` es una cadena, no un diccionario. Necesitas parsearla primero.


Refs:

1. https://www.paradigmadigital.com/dev/uso-de-cadenas-langchain-gen-ai/

2. https://python.langchain.com/v0.1/docs/modules/chains/

3. https://medium.com/@anuragmishra_27746/practical-hands-on-with-langchain-expression-language-lcel-for-building-langchain-agent-chain-2a9364dc4ca3

4. https://medium.com/@itsmybestview/unbridling-the-power-of-langchain-framework-with-lcel-9e5f7bf8af74

5. https://www.youtube.com/playlist?list=PLGPnu4k-KSC1s7apArPZz1AavDJ4Mvitd

6. https://www.youtube.com/watch?v=LzxSY7197ns&t=12s

7. https://www.youtube.com/watch?v=H8DrR2pXbww&list=PLilZ1IiRt0R26etAQeQ9xa5h5UzK18zie&index=3

8. https://www.youtube.com/watch?v=8BV9TW490nQ&t=523s

9. https://www.youtube.com/watch?v=8BV9TW490nQ


Usar esto para localizar
https://www.youtube.com/results?search_query=langchain+lcel+


Uno de los paso de la cadena puede ser una funcion personalizada. Habitualmente para procesar la salida de una forma difrente a los OutParser qu eproporciona LangChain

Vamos a construir una cadena que incluya una funcion personalizada que destaque las palabras de mas de 6 letras del resultado.


In [10]:
from langchain.schema.output_parser import StrOutputParser

# 1. Definimos el prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un traductor profesional."),
    ("human", "Traduce el siguiente texto al inglés: {texto}"),
])

# 2. Definimos el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

# 3. Función personalizada para resaltar palabras clave
def resaltar_palabras_clave(output):
    # Dividimos el texto en palabras
    palabras = output.split()
    # Resaltamos las palabras clave (en este caso, palabras con más de 6 letras)
    texto_resaltado = []
    for palabra in palabras:
        if len(palabra) > 6:
            texto_resaltado.append(f"_{palabra}_")  # Resaltamos con asteriscos
        else:
            texto_resaltado.append(palabra)
    return " ".join(texto_resaltado)

# 4. Creamos la cadena con LCEL: prompt | llm | función personalizada
chain = prompt_template | llm | StrOutputParser() | resaltar_palabras_clave

# 5. Ejecutamos la cadena
texto_original = "La inteligencia artificial está transformando la industria tecnológica."
respuesta = chain.invoke({"texto": texto_original})
display(Markdown(respuesta))

_Artificial_ _intelligence_ is _transforming_ the _technology_ _industry._

StrOutputParser se encarga de extraer el contenido de texto del objeto AIMessage generado por el modelo. Esto elimina la necesidad de acceder manualmente a output.content en la función personalizada.

### **Ejemplo X: Cadena con dos LLMs (Generación y Resumen)**

Vamos a crear una cadena donde un LLM genera contenido y otro LLM resumen ese contenido.

In [13]:
# Primera LLM: Genera contenido
llm1 = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

generate_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Escribe un artículo detallado sobre {topic}."
)

# Segunda LLM: Resumen del contenido generado
llm2 = ChatGoogleGenerativeAI(
    model="gemini-pro",
    api_key=GOOGLE_API_KEY,
    temperature=0.7)

summarize_prompt = PromptTemplate(
    input_variables=["article"],
    template="Resumen el siguiente artículo:\n{article}"
)

# Cadena completa usando LCEL
chain = generate_prompt | llm1 | summarize_prompt | llm2

# Ejecutar la cadena
salida = chain.invoke({"topic": "inteligencia artificial"})
display(Markdown(salida.content))

**Resumen de la Inteligencia Artificial: Transformando el Futuro**

La Inteligencia Artificial (IA) ha evolucionado desde un concepto teórico a una herramienta práctica que transforma diversos aspectos de la vida. Desde su inicio en 1956, la IA ha pasado por fases de optimismo y desilusión. Recientemente, los avances en aprendizaje automático y procesamiento de datos masivos han revivido el interés en la IA.

**Tipos de IA:**
* IA Débil (Narrow AI): Realiza tareas específicas (por ejemplo, asistentes virtuales, sistemas de recomendación).
* IA Fuerte (General AI): Realiza cualquier tarea cognitiva que un humano puede hacer (todavía en desarrollo).

**Aplicaciones de la IA:**
* Salud: Diagnósticos asistidos, personalización de tratamientos.
* Movilidad y Transporte: Vehículos autónomos.
* Finanzas: Detección de fraudes, predicción de tendencias del mercado.
* Marketing y Publicidad: Campañas personalizadas, segmentación de público.
* Agricultura: Optimización de la producción, monitoreo de cultivos.

**Desafíos y Ética:**
* Ética: Los sesgos en los datos de entrenamiento pueden conducir a decisiones injustas.
* Privacidad: La recopilación masiva de datos puede infringir la privacidad.
* Seguridad: La dependencia de la IA en sistemas críticos plantea riesgos de ataques cibernéticos.

**El Futuro de la IA:**
* Integración continua en todos los sectores, mejorando la eficiencia y la productividad.
* Enfoque cuidadoso en la regulación y la gobernanza para garantizar un desarrollo ético.
* Colaboraciones y educación para preparar a las generaciones futuras para un mundo impulsado por la IA.

La IA ofrece un gran potencial para resolver problemas y mejorar la calidad de vida, pero también plantea desafíos que deben abordarse para garantizar su desarrollo beneficioso.

### Reto: Intenta que se visualizen ambos textos. El articulo completo y el resultado

In [ ]:
#Crea aqui tu solucioón

## Solución

Ejemplo X: Integración con API Externa (OpenWeatherMap)

Podemos alimentar al modelo con el resultado de una llamada a una API externa.  
Este ejemplo consulta la API de OpenWeatherMap, obtiene datos meteorológicos y genera un informe personalizado con un LLM.

In [ ]:
import requests

# Función para obtener datos de la API
def fetch_weather(city):
    api_key = "YOUR_OPENWEATHERMAP_API_KEY"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    return response.json()

# LLM: Genera un informe meteorológico personalizado
llm = OpenAI(model_name="text-davinci-003", api_key="YOUR_API_KEY")
weather_prompt = PromptTemplate(
    input_variables=["weather_data"],
    template="Basado en los siguientes datos meteorológicos, proporciona un informe personalizado:\n{weather_data}"
)

# Cadena completa usando LCEL
chain = fetch_weather | weather_prompt | llm

# Ejecutar la cadena
report = chain.invoke({"city": "Madrid"})
print("Informe meteorológico:\n", report)

Ejemplo X: Procesamiento de Contenido de URL

Crea una cadena que obtiene el contenido de una URL, lo procesa con un LLM y guarda el resultado en un archivo.

In [ ]:
import requests
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Función para obtener el contenido de una URL
def fetch_url_content(url):
    response = requests.get(url)
    return response.text

# LLM: Procesa el contenido de la URL
llm = OpenAI(model_name="text-davinci-003", api_key="YOUR_API_KEY")
process_prompt = PromptTemplate(
    input_variables=["content"],
    template="Analiza el siguiente contenido y proporciona una síntesis:\n{content}"
)

# Cadena completa usando LCEL
chain = fetch_url_content | process_prompt | llm

# Ejecutar la cadena y guardar el resultado
synthesis = chain.invoke({"url": "https://es.wikipedia.org/wiki/Inteligencia_artificial"})
with open("synthesis.txt", "w") as file:
    file.write(synthesis)
print("Síntesis guardada en synthesis.txt")

Ejemplo X: Retroalimentacion

La salida de un llm (por ejemplo un texto extenso) se intruce en otro y se le pide qu elo mejore o lo cambien de alguna forma


In [ ]:
# Definir los prompts y el LLM
prompt1 = PromptTemplate(input_variables=["topic"], template="Escribe un resumen sobre {topic}.")
prompt2 = PromptTemplate(input_variables=["summary"], template="Mejora el siguiente texto:\n{summary}")
llm = OpenAI(model_name="text-davinci-003", api_key="YOUR_API_KEY")

# Crear la cadena con retroalimentación
chain = prompt1 | llm | prompt2 | llm

# Ejecutar la cadena
result = chain.invoke({"topic": "inteligencia artificial"})
print(result)